In [1]:
!pip install transformers[sentencepiece] -q 

DATASET DESCRIPTION: using the 20_newsgroups dataset. The dataset used for this task is the 20_newsgroup dataset (link: 20_newsgroup), which contains data, organized into 20 different newsgroups, each corresponding to a different topic.

In [2]:
from sklearn.datasets import fetch_20newsgroups

In [3]:
#Getting the train and test sets #For my purpose I am considering 5 news categories for classification

categories=['alt.atheism','comp.graphics','soc.religion.christian','rec.sport.hockey','sci.space']

train=fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=50)
test=fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=50)

In [5]:
print('Size of train data:',len(train['data']))
print('Size of test data:',len(test['data']))

Size of train data: 2856
Size of test data: 1899


In [6]:
print('Classes',train.target_names)

Classes ['alt.atheism', 'comp.graphics', 'rec.sport.hockey', 'sci.space', 'soc.religion.christian']


In [8]:
#splitting into train test 

x_train=train.data
y_train=train.target
x_test=test.data
y_test=test.target

In [9]:
#Looking at the data

print(train.target_names[y_train[0]])
print("\n ")
print(x_train[0])

sci.space

 
From: thomsonal@cpva.saic.com
Subject: What counntries do space surveillance?
Organization: Science Applications Int'l Corp./San Diego
Lines: 111

      Ethnocentric USian that I am, I've assumed that we and the
xUSSR were the only countries with significant capabilities to track
non-cooperative objects in low Earth orbit. Grazing in a couple of 
databases recently,  I found that Japan has some optical capabilities
along this line, and also uses a radar designed for other purposes
for orbital debris surveys (it isn't clear whether the radar can 
determine orbital elements for the objects it detects). Abstracts of 
the articles are appended. 


    This leads to the more general question: do yet other people than 
the US, Russia, and Japan do space surveillance, and if so, how and 
why? 

Allen Thomson              SAIC                        McLean, VA, USA
-----------------------------------------------------------------------
                         ABSTRACTS

Optical t

ktrain - ktrain is a Python library that provides an easy-to-use interface for building, training, and deploying deep learning models. It is built on top of the popular deep learning library TensorFlow and is designed to simplify the process of building and training deep learning models.

ktrain includes pre-processing tools for text and image data, as well as pre-built models for text classification, sequence labeling, image classification, and more. It also provides support for transfer learning, allowing you to fine-tune pre-trained models on your own data.

In [11]:
pip install ktrain 

Note: you may need to restart the kernel to use updated packages.


In [12]:
# Defining our model
model_name='distilbert-base-uncased'

In [13]:
import ktrain
from ktrain import text

In [14]:
t=text.Transformer(model_name, maxlen=500, classes=train.target_names)

C:\Users\lenovo\anaconda3\lib\site-packages\ktrain\text\preprocessor.py:382: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn(


In [15]:
#ktrain pre-processes the data according to the model

pr_train=t.preprocess_train(x_train,y_train)
pr_test=t.preprocess_test(x_test,y_test)

preprocessing train...
language: en
train sequence lengths:
	mean : 309
	95percentile : 886
	99percentile : 1913


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 321
	95percentile : 861
	99percentile : 2155


In [16]:
#we define a classifier with pretrained weights and randomly initialized final layers that can be fine-tuned. 
#The model will be wrapped in a ktrain Learner object that will allow us to easily train and inspect the model and 
#use it to make predictions on new data. This is a part of fine-tuning the model as per our requirements


model=t.get_classifier()
learner=ktrain.get_learner(model, train_data=pr_train, val_data=pr_test, batch_size=4)

In [ ]:
#For BERT-based models, learning rates between 2e-5 and 5e-5 generally work well across a wide range of datasets.

In [17]:
learner.fit_onecycle(5e-5,2)



begin training using onecycle policy with max lr of 5e-05...
Epoch 1/2
714/714 [==============================] - 5877s 8s/step - loss: 0.4752 - accuracy: 0.8701 - val_loss: 0.4760 - val_accuracy: 0.8520
Epoch 2/2
714/714 [==============================] - 5348s 7s/step - loss: 0.0981 - accuracy: 0.9744 - val_loss: 0.1557 - val_accuracy: 0.9558


In [24]:
#Accuracy is pretty impressive in just 2 epochs

In [18]:
#Declaring ktrain predictor

predictor = ktrain.get_predictor(learner.model, preproc=t)

In [19]:
predictor.predict('Jesus Christ is the central figure of Christianity.')

1/1 [==============================] - 4s 4s/step


'soc.religion.christian'

In [20]:
predictor.predict('Mars is one of earths neighbour out there in space')

1/1 [==============================] - 0s 99ms/step


'sci.space'

In [21]:
predictor.save('/tmp/my_distilbert_predictor')

In [22]:
#reloading the model

reloaded_predictor = ktrain.load_predictor('/tmp/my_distilbert_predictor')

In [23]:
reloaded_predictor.predict('Blender is the future of 3D modelling')

1/1 [==============================] - 3s 3s/step


'comp.graphics'